In [1]:
import os
import json
import pandas as pd

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
key=os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key=key , model_name='gpt-3.5-turbo', temperature=0.5)

y:\ml_projects\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "option": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "option": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "option": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
}

In [8]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text it is your job to create a quiz of {number} multiple choice questions for {subject} \
students in {tone} tone. Make sure the questions are not repeated and check all questions are conforming to the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [11]:
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students, you need to evaluate the complexity of
the question and give a complete analysis of the quiz. Only use at max 50 words for complexity. If the quiz is not at par with the cognitive and 
analytical abilities of the students, update the quiz questions that need to be changed and change the tone such that it perfectly fits 
the student's ability
Quiz MCQs:
{quiz}
Check from an expert English writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(input_variables=['subject', 'quiz'], template=TEMPLATE2)

In [13]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [14]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
                                        output_variables = ["quiz", "review"], verbose=True)

In [15]:
file_path = "Y:\ml_projects\mcqgen\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()

In [16]:
TEXT

"A robot is a machine—especially one programmable by a computer—capable of carrying out a complex series of actions\nautomatically.A robot can be guided by an external control\ndevice, or the control may be embedded within. Robots may be\nconstructed to evoke human form, but most robots are taskperforming machines, designed with an emphasis on stark\nfunctionality, rather than expressive aesthetics.\nRobots can be autonomous or semi-autonomous and range\nfrom humanoids such as Honda's Advanced Step in Innovative\nMobility (ASIMO) and TOSY's TOSY Ping Pong Playing Robot\n(TOPIO) to industrial robots, medical operating robots, patient\nassist robots, dog therapy robots, collectively programmed\nswarm robots, UAV drones such as General Atomics MQ-1\nPredator, and even microscopic nano robots. By mimicking a\nlifelike appearance or automating movements, a robot may\nconvey a sense of intelligence or thought of its own.\nAutonomous things are expected to proliferate in the future,\nwith hom

In [17]:
# Serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
NUMBER = 5
SUBJECT = 'Robots'
TONE = 'simple'

In [19]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking/
# How to setup Token Usage Tracking in Langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

y:\ml_projects\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:A robot is a machine—especially one programmable by a computer—capable of carrying out a complex series of actions
automatically.A robot can be guided by an external control
device, or the control may be embedded within. Robots may be
constructed to evoke human form, but most robots are taskperforming machines, designed with an emphasis on stark
functionality, rather than expressive aesthetics.
Robots can be autonomous or semi-autonomous and range
from humanoids such as Honda's Advanced Step in Innovative
Mobility (ASIMO) and TOSY's TOSY Ping Pong Playing Robot
(TOPIO) to industrial robots, medical operating robots, patient
assist robots, dog therapy robots, collectively programmed
swarm robots, UAV drones such as General Atomics MQ-1
Predator, and even microscopic nano robots. By mimicking a
lifelike appearance or automating movements, a robot may
convey a sense of intelligence 

In [20]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 1951
Prompt Tokens: 1503
Completion Tokens: 448
Total Cost: 0.0031505


In [21]:
response

{'text': "A robot is a machine—especially one programmable by a computer—capable of carrying out a complex series of actions\nautomatically.A robot can be guided by an external control\ndevice, or the control may be embedded within. Robots may be\nconstructed to evoke human form, but most robots are taskperforming machines, designed with an emphasis on stark\nfunctionality, rather than expressive aesthetics.\nRobots can be autonomous or semi-autonomous and range\nfrom humanoids such as Honda's Advanced Step in Innovative\nMobility (ASIMO) and TOSY's TOSY Ping Pong Playing Robot\n(TOPIO) to industrial robots, medical operating robots, patient\nassist robots, dog therapy robots, collectively programmed\nswarm robots, UAV drones such as General Atomics MQ-1\nPredator, and even microscopic nano robots. By mimicking a\nlifelike appearance or automating movements, a robot may\nconvey a sense of intelligence or thought of its own.\nAutonomous things are expected to proliferate in the future,\

In [22]:
quiz = response.get("quiz")

In [26]:
quiz = json.loads(quiz)

In [28]:
quiz

{'1': {'mcq': 'What was the first commercial, digital, and programmable robot called?',
  'option': {'a': 'ASIMO', 'b': 'TOPIO', 'c': 'Unimate', 'd': 'Predator'},
  'correct': 'c'},
 '2': {'mcq': "Who invented the term 'robot'?",
  'option': {'a': 'Karel Čapek',
   'b': 'Josef Čapek',
   'c': 'William Grey Walter',
   'd': 'George Devol'},
  'correct': 'b'},
 '3': {'mcq': 'What is the branch of technology that deals with the design, construction, and application of robots?',
  'option': {'a': 'Biology',
   'b': 'Robotics',
   'c': 'Astronomy',
   'd': 'Geology'},
  'correct': 'b'},
 '4': {'mcq': 'Which robot was used by General Motors in 1961 to lift pieces of hot metal?',
  'option': {'a': 'ASIMO', 'b': 'TOPIO', 'c': 'Unimate', 'd': 'Predator'},
  'correct': 'c'},
 '5': {'mcq': 'What was the first electronic autonomous robot created in 1948?',
  'option': {'a': 'ASIMO', 'b': 'TOPIO', 'c': 'Unimate', 'd': 'Predator'},
  'correct': 'c'}}

In [29]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value['option'].items()
        ]
    )
    correct = value['correct']
    quiz_table_data.append({'MCQ': mcq, 'Choices': options, 'Correct': correct})

In [30]:
quiz_table_data

[{'MCQ': 'What was the first commercial, digital, and programmable robot called?',
  'Choices': 'a: ASIMO | b: TOPIO | c: Unimate | d: Predator',
  'Correct': 'c'},
 {'MCQ': "Who invented the term 'robot'?",
  'Choices': 'a: Karel Čapek | b: Josef Čapek | c: William Grey Walter | d: George Devol',
  'Correct': 'b'},
 {'MCQ': 'What is the branch of technology that deals with the design, construction, and application of robots?',
  'Choices': 'a: Biology | b: Robotics | c: Astronomy | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which robot was used by General Motors in 1961 to lift pieces of hot metal?',
  'Choices': 'a: ASIMO | b: TOPIO | c: Unimate | d: Predator',
  'Correct': 'c'},
 {'MCQ': 'What was the first electronic autonomous robot created in 1948?',
  'Choices': 'a: ASIMO | b: TOPIO | c: Unimate | d: Predator',
  'Correct': 'c'}]

In [32]:
quiz_df = pd.DataFrame(quiz_table_data)

In [33]:
quiz_df.to_csv("robot.csv", index=False)